In [1]:
import os
import warnings

import numpy as np
import pytest

from gymnasium import utils
from gymnasium.envs.mujoco import MujocoEnv
from gymnasium.error import Error
from gymnasium.spaces import Box
from gymnasium.utils.env_checker import check_env

In [2]:
class QRPfRA_v3(MujocoEnv, utils.EzPickle):
    metadata = {
        "render_modes": [
            "human",
            "rgb_array",
            "depth_array",
        ],
    }

    def __init__(self, xml_file="/Users/deniz/PycharmProjects/QRPfRA_Senior_Project/QRPfRA_sim_body_STLs/qrpfra_v3_leg_ik_scene.xml", frame_skip=1, **kwargs):
        utils.EzPickle.__init__(self, xml_file, frame_skip, **kwargs)

        MujocoEnv.__init__(
            self,
            xml_file,
            frame_skip=frame_skip,
            observation_space=None,  # needs to be defined after
            default_camera_config={},
            **kwargs,
        )

        self.metadata = {
            "render_modes": [
                "human",
                "rgb_array",
                "depth_array",
            ],
            "render_fps": int(np.round(1.0 / self.dt)),
        }

        obs_size = self.data.qpos.size + self.data.qvel.size

        self.observation_space = Box(
            low=-np.inf, high=np.inf, shape=(obs_size,), dtype=np.float64
        )

    def step(self, action):
        x_position_before = self.data.qpos[0]

        self.do_simulation(action, self.frame_skip)
        x_position_after = self.data.qpos[0]

        observation = self._get_obs()
        reward = x_position_after - x_position_before
        info = {}

        if self.render_mode == "human":
            self.render()

        return observation, reward, False, False, info

    def _get_obs(self):
        #position = self.data.qpos.flat.copy()
        #velocity = self.data.qvel.flat.copy()
        sensor_data = self.data.sensordata.flat.copy()

        return sensor_data

    def reset_model(self):
        qpos = self.init_qpos
        qvel = self.init_qvel
        self.set_state(qpos, qvel)

        observation = self._get_obs()

        return observation

    def _get_reset_info(self):
        return {"works": True}



In [3]:
env = QRPfRA_v3()
check_env(env)
obs = env.reset()
print(obs)


(array([-0.045 ,  0.1335, -0.107 ,  0.    ,  0.    ,  0.    ]), {'works': True})


/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:247: UserWarning: WARN: For Box action spaces, we recommend using a symmetric and normalized space (range=[-1, 1] or [0, 1]). See https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html for more information.
  logger.warn(
/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:225: UserWarning: WARN: A Box observation space minimum value is -infinity. This is probably too low.
  logger.warn(
/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:229: UserWarning: WARN: A Box observation space maximum value is -infinity. This is probably too high.
  logger.warn(
/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:321: UserWarning: WARN: Not able to test alternative render modes due to the environ

In [4]:
left_leg_dataset = [] # [obs0, obs1, obs2, act0, act1, act2]

In [6]:
for i in range(-100, 100):
    for j in range(-100, 100):
        for k in range(-100, 100):
            #env.render_mode = "human"

            obs, reward, done, _, info = env.step([float(m) for m in [i, j, k]])
            left_leg_dataset.append([obs[0], obs[1], obs[2], i/100, j/100, k/100])

            #print(obs)
            if done:
                env.reset()

env.close()

In [9]:
print(len(left_leg_dataset))

8238193


In [10]:
#Convert left leg list to numpy array
left_leg_dataset_as_np = np.array(left_leg_dataset)

In [29]:
print(left_leg_dataset_as_np[0:4])

observations = left_leg_dataset_as_np[:, 0:3][:]
print(observations)
actions = left_leg_dataset_as_np[:, 3:][:]
print(actions)

[[-0.045       0.1335     -0.107      -1.         -1.         -1.        ]
 [ 0.00872301  0.08305386 -0.17710357 -1.         -1.         -0.99      ]
 [ 0.10444321 -0.03413705 -0.18731909 -1.         -1.         -0.98      ]
 [ 0.13397228 -0.11299516 -0.14351767 -1.         -1.         -0.97      ]]
[[-0.045       0.1335     -0.107     ]
 [ 0.00872301  0.08305386 -0.17710357]
 [ 0.10444321 -0.03413705 -0.18731909]
 ...
 [ 0.02371278  0.04512389  0.07646926]
 [ 0.02321392  0.04400059  0.07617783]
 [ 0.02270487  0.04288343  0.07588045]]
[[-1.   -1.   -1.  ]
 [-1.   -1.   -0.99]
 [-1.   -1.   -0.98]
 ...
 [ 0.99  0.99  0.97]
 [ 0.99  0.99  0.98]
 [ 0.99  0.99  0.99]]


In [45]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train_obs, test_obs, train_actions, test_actions = train_test_split(observations, actions, test_size=0.1)

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(3, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='tanh')
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Train the model
model.fit(train_obs, train_actions, epochs=3, batch_size=32)

# Evaluate the model on the testing set
test_loss = model.evaluate(test_obs, test_actions)
print('Test Loss:', test_loss)


Epoch 1/3
231700/231700 [==============================] - 119s 513us/step - loss: 0.0274 - accuracy: 0.9151
Epoch 2/3
231700/231700 [==============================] - 121s 524us/step - loss: 0.0237 - accuracy: 0.9258
Epoch 3/3
25745/25745 [==============================] - 9s 344us/step - loss: 0.0221 - accuracy: 0.9364
Test Loss: [0.022083459421992302, 0.9363526105880737]


In [46]:
model.save("left_legs_model")

INFO:tensorflow:Assets written to: left_legs_model/assets


INFO:tensorflow:Assets written to: left_legs_model/assets
